# Generating training data on the ODC <img align="right" src="../Supplementary_data/DE_Africa_Logo_Stacked_RGB_small.jpg">

* **Products used:** 
[ga_ls8c_gm_2_annual](https://explorer.digitalearth.africa/ga_ls8c_gm_2_annual)


## Background

**Training data** is the most important part of any supervised machine learning workflow. The quality of the training data has a greater impact on the classification than the algorithm used. Large and accurate training data sets are preferable: increasing the training sample size results in increased classification accuracy ([Maxell et al 2018](https://www.tandfonline.com/doi/full/10.1080/01431161.2018.1433343)).

A review of training data methods in the context of Earth Observation is available [here](https://www.mdpi.com/2072-4292/12/6/1034) 

When creating training data, be sure to capture the **spectral variability** of the class, and to use imagery from the time period you want to classify (rather than relying on basemap composites). Another common problem with training data is **class imbalance**. This can occur when one of your classes is relatively rare and therefore the rare class will comprise a smaller proportion of the training set. When imbalanced data is used, it is common that the final classification will under-predict less abundant classes relative to their true proportion.

There are many platforms to use for gathering training data, the best one to use depends on your application. GIS platforms are great for collection training data as they are highly flexible platforms; [geo-wiki](https://www.geo-wiki.org/) and [Collect Earth Online](https://collect.earth/home) are two open-source websites that also very useful.



## Description
This notebook will extract training data from the `open-data-cube` using geometries within a shapefile (or geojson). To do this, we rely on a custom `deafrica-sandbox-notebooks` function called `collect_training_data`, contained within the [deafrica_classificationtools](../Scripts/deafrica_classificationtools.py) script.  The goal of this notebook is to familarise users with this function so they can extract the appropriate data for their use-case.

1. First we do this
2. Then we do this
3. Finally we do this

***

## Getting started

To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell. 

### Load packages


In [1]:
%matplotlib inline

import sys
import os
import xarray as xr
import subprocess as sp
import numpy as np
import geopandas as gpd

sys.path.append('../Scripts')
from deafrica_classificationtools import collect_training_data 
from deafrica_bandindices import calculate_indices
from deafrica_temporal_statistics import temporal_statistics
from deafrica_plotting import map_shapefile

/env/lib/python3.6/site-packages/datacube/storage/masking.py:4: DeprecationWarning: datacube.storage.masking has moved to datacube.utils.masking
  category=DeprecationWarning)


## Analysis parameters

* `path`: The path to the input shapefile from which we will extract training data. A default shapefile is provided.
* `field`: This is the name of column in your shapefile attribute table that contains the class labels. The class labels must be integers
* `ncpus`: Set this value to > 1 to parallize the collection of training data. eg. npus=8

In [2]:
path = 'data/training/test_training_dataset.shp' 
field = 'class'

# automatically detect number of cpus, adjust to [-4:] if working on XXL Sandbox
ncpus= int(float(sp.getoutput('env | grep CPU')[-3:]))

print('ncpus = '+str(ncpus))

ncpus = 2


## Preview input data

We can load and preview our input data shapefile using `geopandas`. The shapefile should contain a column with class labels (e.g. 'class'). These labels will be used to train our model. Remember, the class labels must be represented by integers.


In [3]:
# Load input data shapefile
input_data = gpd.read_file(path)

# Plot first five rows
input_data.head()

,class,ID,CODE,COUNTRY,geometry
0,1,21,BOT,Southern,"POLYGON ((19.76605 -34.71434, 19.76605 -34.713..."
1,1,21,BOT,Southern,"POLYGON ((20.17941 -34.21388, 20.17941 -34.213..."
2,0,21,BOT,Southern,"POLYGON ((21.35220 -33.78466, 21.35220 -33.783..."
3,0,21,BOT,Southern,"POLYGON ((25.72613 -33.69673, 25.72613 -33.695..."
4,0,21,BOT,Southern,"POLYGON ((21.78891 -33.53318, 21.78891 -33.532..."


In [4]:
# Plot training data in an interactive map
map_shapefile(input_data, attribute=field)

Label(value='')

Map(center=[-22.12386984762135, 26.34933471312903], controls=(ZoomControl(options=['position', 'zoom_in_text',…

## Collect training data

The function `collect_training_data` takes our shapefile containing class labels and extracts training data from the datacube over the location specified by the input geometries. The function will also pre-process our training data by stacking the arrays into a useful format and removing an `NaN` (not-a-number) values. 

`Collect_training_data` has the ability to generate many different types of **feature layers**, relatively simple layers can be calculated using pre-defined parameters within the function; more complex layers can be computed by passing in a `custom_func`. To begin with, let's try generating feature layers using the pre-defined methods.

The in-built feature layer parameters are described below:
* `product`: The name of the product to extract from the datacube. In this example we use a geomedian composite from 2018, `'ga_ls8c_gm_2_annual'`
* `time`: The time range over to extract data
* `calc_indices`: This parameter provides a method for calculating a number of remote sensing indices (e.g. `['NDWI', 'NDVI']`). Any of the indices found in the [deafrica_bandindices](../Scripts/deafrica_bandindices.py) script can be used here
* `reduce_func`: Function to reduce the data from multiple time steps to a single timestep. Options are 'mean', 'median', 'std', 'max', 'min', 'geomedian'. In the default example we are loading a geomedian composite, so there is not time dimension to reduce.
* `drop`: If this variable is set to True, and 'calc_indices' are supplied, the spectral bands will be dropped from the dataset leaving only the band indices as data variables in the dataset.
* `zonal_stats`: An optional string giving the names of zonal statistics to calculate for each polygon. Default is None (all pixel values are returned). Supported values are 'mean', 'median', 'max', 'min', and 'std'.

In addition to the parameters required for `collect_training_data`, we also need to set up a few parameters for the open-data-cube query, such as `measurements` (the bands to load from the satellite), the `resolution` (the cell size), and the `output_crs` (the output projection). 

In [5]:
#set up our inputs to collect_training_data
products =  ['ga_ls8c_gm_2_annual']
time = ('2018')
reduce_func = None
calc_indices = ['EVI', 'LAI'] 
drop = False
zonal_stats = 'median' 

# Set up the inputs for the ODC ODC query
measurements =  ['blue','green','red','nir','swir_1','swir_2']
resolution = (-20,20)
output_crs='epsg:6933'

In [6]:
#generate a datacube query object
query = {
    'time': time,
    'measurements': measurements,
    'resolution': resolution,
    'output_crs': output_crs,
    'group_by' : 'solar_day',
}

> **Note**: With supervised classification, its common to have many, many labelled geometries in the training data. `collect_training_data` can parallelize across the geometries in order to speed up the extracting of training data. Setting `ncpus>1` will automatically trigger the parallelization, however, its best to set `ncpus=1` to begin with to assist with debugging before triggering the parallelization. 

Now let's run the `collect_training_data` function.

In [7]:
column_names, model_input = collect_training_data(
                                    gdf=input_data,
                                    products=products,
                                    dc_query=query,
                                    ncpus=ncpus,
                                    field=field,
                                    calc_indices=calc_indices,
                                    reduce_func=reduce_func,
                                    drop=drop,
                                    zonal_stats=zonal_stats)

Calculating indices: ['EVI', 'LAI']
Taking zonal statistic: median


100%|██████████| 292/292 [04:40<00:00,  1.04it/s]


Output training data has shape (292, 9)
Removed NaNs, cleaned input shape:  (292, 9)


The function returns two numpy arrays, the first (`column_names`) contains a list of the names of the feature layers we've computed:

In [8]:
print(column_names)

['class', 'blue', 'green', 'red', 'nir', 'swir_1', 'swir_2', 'EVI', 'LAI']


The second array (`model_input`) contains the data from our labelled geometries. The first item in the array is the class integer (e.g. in the default example 1. or 0.), the second set of items are the values for each feature layer we computed:

In [9]:
print(np.array_str(model_input, precision=2, suppress_small=True))

[[ 1.    0.11  0.18 ...  0.35  0.   -0.12]
 [ 1.    0.07  0.09 ...  0.23  0.   -0.12]
 [ 0.    0.03  0.05 ...  0.13  0.   -0.12]
 ...
 [ 1.    0.06  0.1  ...  0.24  0.   -0.12]
 [ 0.    0.05  0.08 ...  0.16  0.   -0.12]
 [ 0.    0.04  0.06 ...  0.14  0.   -0.12]]


## Custom feature layers



## Export training data



In [ ]:
# output_file = "training_data.txt"
# np.savetxt(output_file, model_input, header=" ".join(column_names), fmt="%4f")

In [ ]:
# filename = os.path.join(working_dir, '2010_2015_median_indices_training_data_binary.txt')
# model_input = numpy.loadtxt(filename, skiprows=1)
# random_state = 1234
# # Set up header and input features
# with open(filename, 'r') as file:
#     header = file.readline()
# column_names = header.split()
# column_names_indices = {}
# for col_num, var_name in enumerate(column_names):
#     column_names_indices[var_name] = col_num
# model_variables = ['blue','red','green','nir','swir1','swir2','edev','sdev','bcdev', 'NDVI', 'MNDWI', 'BAI', 'BUI', 'BSI', 'TCG', 'TCW', 'TCB', 'NDMI', 'LAI', 'EVI', 'AWEI_sh', 'BAEI', 'NDSI', 'SAVI']
# model_col_indices = []
# for model_var in model_variables:
#     model_col_indices.append(column_names_indices[model_var])

## Recommended next steps

To continue working through the notebooks in this `Scalable Machine Learning on the ODC` workflow, go to the next notebook `Training data knowledge base`

1. **1_Generate_training_data (this notebook)**
2. [Training data knowledge base]()


***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Africa data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks).

**Last modified:** August 2020

**Compatible datacube version:** 

In [ ]:
print(datacube.__version__)

## Tags
Browse all available tags on the DE Africa User Guide's [Tags Index](https://) (placeholder as this does not exist yet)